In [20]:
# Import libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [21]:
def create_url(league_name,league_id):
    url = f'https://www.transfermarkt.com/{league_name}/transfers/wettbewerb/{league_id}/plus/?saison_id=2023&s_w=s'
    return(url)

In [22]:
def create_club_names(doc):  
    mydivs = doc.find_all("div", {"class": "box"})
    club_names = []
    for eachDiv in mydivs:
        try:
            club_names.append(eachDiv.find_all('h2')[0].text.strip())
            club_names.append(eachDiv.find_all('h2')[0].text.strip())
        except:
            pass
        
    club_names = club_names[4:len(club_names)-2]
    return club_names

In [23]:
def createDataframe(read_html_pandas_data, club_names):    
    appendData = []
    for x in range(1, len(read_html_pandas_data)):
        df1 = pd.DataFrame()
        df1 = read_html_pandas_data[x]
        df1['club_name'] = club_names[x-1]
        df1 = df1.rename(columns={'Left.1': 'club_involved_name'})
        df1 = df1.rename(columns={'Joined.1': 'club_involved_name'})
        appendData.append(df1)
    appended_data = pd.concat(appendData)

    appended_data.loc[appended_data['In'].isnull(), 'transfer_movement'] = 'Out' 
    appended_data.loc[appended_data['Out'].isnull(), 'transfer_movement'] = 'In' 

    appended_data['player_name'] = appended_data['Out'].fillna(appended_data['In'])
    appended_data = appended_data.drop(columns=['In', 'Out', 'Nat.', 'Left', 'Joined'])

    appended_data = appended_data.iloc[:, [6,8,0,1,2,3,4,5,7]]

    
    appended_data['fee_cleaned'] = appended_data['Fee']
    appended_data.loc[~appended_data['fee_cleaned'].str.contains('€', na=False),'fee_cleaned'] = np.nan
    appended_data['fee_cleaned'] = appended_data['fee_cleaned'].str.replace('Loan fee:', '')    
    return appended_data


In [24]:
league_df = pd.read_csv('config/league-meta.csv')
league_name = league_df.league_name.values.tolist()
league_id = league_df.league_id.values.tolist()

for league in range(0,len(league_id)):
    atd_url = create_url(league_name[league], league_id[league])
    r = requests.get(atd_url,headers ={'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36'})
    doc = BeautifulSoup(r.text)
    read_html_pandas_data = pd.read_html(r.text)
    club_names = create_club_names(doc)
    print(atd_url)
    print(f'Total tables: {len(read_html_pandas_data)}')
    print("League Count:", len(club_names))
    
    df = createDataframe(read_html_pandas_data, club_names)

    df.to_csv(f"data/{league_name[league]}.csv")

https://www.transfermarkt.com/premier-league/transfers/wettbewerb/GB1/plus/?saison_id=2023&s_w=s
Total tables: 41
League Count: 40
https://www.transfermarkt.com/championship/transfers/wettbewerb/GB2/plus/?saison_id=2023&s_w=s
Total tables: 49
League Count: 48
https://www.transfermarkt.com/primera-division/transfers/wettbewerb/ES1/plus/?saison_id=2023&s_w=s
Total tables: 41
League Count: 40
https://www.transfermarkt.com/1-bundesliga/transfers/wettbewerb/L1/plus/?saison_id=2023&s_w=s
Total tables: 37
League Count: 36
https://www.transfermarkt.com/serie-a/transfers/wettbewerb/IT1/plus/?saison_id=2023&s_w=s
Total tables: 41
League Count: 40
https://www.transfermarkt.com/ligue-1/transfers/wettbewerb/FR1/plus/?saison_id=2023&s_w=s
Total tables: 37
League Count: 36
https://www.transfermarkt.com/liga-nos/transfers/wettbewerb/PO1/plus/?saison_id=2023&s_w=s
Total tables: 37
League Count: 36
https://www.transfermarkt.com/eredivisie/transfers/wettbewerb/NL1/plus/?saison_id=2023&s_w=s
Total tables: